In [ ]:
import torch
from src import models # Import kiến trúc model của bạn
import cfg

# --- Tải mô hình đã được tỉa và fine-tune ---
# Đảm bảo bạn đã thực hiện bước prune.remove() để mô hình không còn mask
model = models.Deeper1DCNN(input_channels=1, num_classes=3)
# Giả sử bạn đã lưu state_dict sau khi remove mask
checkpoint = torch.load('./outputs/models/test.pth', map_location=torch.device('cpu')) # Dùng map_location để đảm bảo chạy được trên CPU
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()
model.cpu() # Chuyển về CPU trước khi export

# --- Export sang ONNX ---
onnx_file_path = "./outputs/models/test.onnx"
dummy_input = torch.randn(1, 1, cfg.SAMPLE_LENGTH) # Batch size = 1

torch.onnx.export(
    model,
    dummy_input,
    onnx_file_path,
    input_names=["x"], output_names=["logits"],
    dynamic_axes={"x": {0: "N", 2: "L"}, "logits": {0: "N"}},
    opset_version=13, do_constant_folding=True
)

print(f"Đã export thành công mô hình sang {onnx_file_path}")

/tmp/ipykernel_1834943/3962559205.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./outputs/models/test.pth', map_location=torch.device('cpu')) 

Đã export thành công mô hình sang ./outputs/models/test.onnx


In [2]:
if cfg.IMBALANCED_MODE == True: 
    X_train, Y_train, X_val, Y_val, X_test, Y_test = data_loader.data_import(cfg.OVERLAPPING_RATIO,cfg.BASE_PATH,cfg.SAMPLE_LENGTH, cfg.PREPROCESSING)
    X_train_imbalanced, Y_train_imbalanced = data_loader.create_imbalanced_data(X_train, Y_train, cfg.IMBALANCED_RATIO, cfg.NUM_CLASSES)
    #X_train_normalized, X_val_normalized, X_test_normalized = data_loader.normalize_data(X_train_imbalanced, X_val, X_test)
    X_train_smoted, Y_train_smoted = data_loader.apply_smote(X_train_imbalanced, Y_train_imbalanced, cfg.RANDOM_STATE, cfg.SAMPLE_LENGTH)
    train_loader, val_loader, test_loader = data_loader.create_dataloaders(X_train_smoted, Y_train_smoted,
                                                                            X_val, Y_val,
                                                                            X_test, Y_test,
                                                                            cfg.SAMPLE_LENGTH, cfg.CNN1D_INPUT, cfg.BATCH_SIZE, cfg.RANDOM_STATE)
else:
    X_train, Y_train, X_val, Y_val, X_test, Y_test = data_loader.data_import(cfg.OVERLAPPING_RATIO,cfg.BASE_PATH,cfg.SAMPLE_LENGTH,cfg.PREPROCESSING)
    X_train_normalized, X_val_normalized, X_test_normalized = data_loader.normalize_data(X_train, X_val, X_test)
    train_loader, val_loader, test_loader = data_loader.create_dataloaders(X_train, Y_train,
                                                                            X_val, Y_val,
                                                                            X_test, Y_test,
                                                                            cfg.SAMPLE_LENGTH, cfg.CNN1D_INPUT, cfg.BATCH_SIZE, cfg.RANDOM_STATE)

>>> Importing: Record = '97', Key = 'DE'
    [INFO] Label: Normal
    [INFO] Size: 119
>>> Importing: Record = '97', Key = 'FE'
    [INFO] Label: Normal
    [INFO] Size: 119
>>> Importing: Record = '98', Key = 'DE'
    [INFO] Label: Normal
    [INFO] Size: 236
>>> Importing: Record = '209', Key = 'FE'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '210', Key = 'DE'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '278', Key = 'DE'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '278', Key = 'FE'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '280', Key = 'DE'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '280', Key = 'BA'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '271', Key = 'DE'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '271', Key = 'FE'
    [INFO] Label: IR
    [INFO] Size: 59
>>> Importing: Record = '276', Key = 'BA'
    [INFO] Label: IR
    [INFO] S

In [4]:
import numpy as np

# train_loader = ... (tải train_loader của bạn)
calibration_data = []
num_calibration_batches = 5 # Ví dụ: 5 batches

for i, (data, target) in enumerate(train_loader):
    if i >= num_calibration_batches:
        break
    calibration_data.append(data.numpy())

# Gộp các batch lại và lưu
calibration_data = np.concatenate(calibration_data, axis=0)
np.save("calibration_data.npy", calibration_data)

print(f"Đã lưu {calibration_data.shape[0]} mẫu dữ liệu hiệu chỉnh vào calibration_data.npy")

Đã lưu 640 mẫu dữ liệu hiệu chỉnh vào calibration_data.npy


In [5]:
import numpy as np
print("Bắt đầu quá trình trích xuất dữ liệu từ DataLoader...")

all_features = []
all_labels = []

# Vô hiệu hóa tính toán gradient để tăng tốc độ và giảm bộ nhớ
with torch.no_grad():
    for data_batch, labels_batch in test_loader:
        # Chuyển tensor về CPU (nếu đang dùng GPU) rồi sang NumPy
        # và thêm vào danh sách
        all_features.append(data_batch.cpu().numpy())
        all_labels.append(labels_batch.cpu().numpy())

print("Đã trích xuất xong!")

# Bước 4: Nối các batch lại thành một mảng NumPy lớn duy nhất
# np.concatenate sẽ ghép các mảng trong list theo trục đã cho (mặc định là 0)
final_features_array = np.concatenate(all_features, axis=0)
final_labels_array = np.concatenate(all_labels, axis=0)

print(f"Kích thước mảng features cuối cùng: {final_features_array.shape}")
print(f"Kích thước mảng labels cuối cùng: {final_labels_array.shape}")

# Bước 5: Lưu các mảng thành file .npy
# Nên lưu features và labels thành 2 file riêng biệt
np.save('X_test.npy', final_features_array)
np.save('Y_test.npy', final_labels_array)

print("Đã lưu dữ liệu thành công vào file 'features_data.npy' và 'labels_data.npy'")

# --- (Tùy chọn) Kiểm tra lại bằng cách tải file ---
print("\nKiểm tra lại file đã lưu:")
loaded_features = np.load('X_test.npy')
loaded_labels = np.load('Y_test.npy')

print(f"Kích thước features đã tải: {loaded_features.shape}")
print(f"Kích thước labels đã tải: {loaded_labels.shape}")

# Kiểm tra xem dữ liệu có khớp không
assert np.array_equal(final_features_array, loaded_features)
assert np.array_equal(final_labels_array, loaded_labels)
print("Kiểm tra thành công, dữ liệu khớp!")

Bắt đầu quá trình trích xuất dữ liệu từ DataLoader...
Đã trích xuất xong!
Kích thước mảng features cuối cùng: (1239, 1, 2048)
Kích thước mảng labels cuối cùng: (1239,)
Đã lưu dữ liệu thành công vào file 'features_data.npy' và 'labels_data.npy'

Kiểm tra lại file đã lưu:
Kích thước features đã tải: (1239, 1, 2048)
Kích thước labels đã tải: (1239,)
Kiểm tra thành công, dữ liệu khớp!
